## Rafael Larrazolo de la Cruz
## 118426

## Ejercicio/Tarea

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

In [17]:
from dask import dataframe
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from dask import delayed

### Lectura de Datos y Modificación de Variables
Las variables que contienen el tiempo en que inicia y termina el viaje son convertidas a tipo `datetime`.

Se calcula la variable a predecir `prop` que es la proporción que representa la propina del monto cobrado por el viaje.

In [2]:
trips_df = dataframe.read_csv("/data/trips.csv")
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
trips_df["prop"] = trips_df["tip_amount"]/trips_df["fare_amount"]
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,prop
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.117647
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221333


Se modifica la tabla quitando las observaciones en las que no hubo cobro `fare_amount`.

Se crea la variable `trip_time` la cual indica el tiempo en segundos que dura el trayecto.

In [3]:
df = trips_df.compute()
df = df[df["fare_amount"]>0]
df["pickup_hour"] = df["tpep_pickup_datetime"].apply(lambda x: x.hour)
df["trip_time"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
df["trip_time"] = df["trip_time"].apply(lambda x:int(x.seconds))
label_encoder = LabelEncoder()
df["car_type"] = label_encoder.fit_transform(df['car_type'])
df = df.drop(["tpep_pickup_datetime", "tpep_dropoff_datetime", "taxi_id"], axis= 1)
df.head()

,car_type,fare_amount,passenger_count,tip_amount,trip_distance,prop,pickup_hour,trip_time
0,0,22.0,1,4.60,6.90,0.209091,1,1170
1,0,9.0,1,0.00,1.81,0.000000,23,587
2,0,7.5,1,1.00,0.96,0.133333,15,627
3,0,8.5,1,1.00,1.90,0.117647,8,551
4,0,7.5,1,1.66,1.00,0.221333,12,568


### Modelo

Se separan los datos en entrenamiento y prueba.

In [4]:
y = df["prop"]
X = df.drop("prop", axis = 1)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.70)

En primera instancia propuse el modelo `MLPRegressor` para hacer la predicción de la variable `prop`.

In [7]:
model = MLPRegressor(solver='adam')

Creo mi `pipeline` el cual contiene al modelo mencionado y previamente un módulo que escala las variables utilizadas para realizar la predicción.

In [10]:
my_pipe = Pipeline([('my_scaler', StandardScaler()), ('mlp', MLPRegressor(solver='adam'))])
my_pipe.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('my_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_r...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))])

Construyo mi `grid` con los hiperparámetros a probar para el modelo. 

Se prueban con varios nivels de capas ocultas, funciones de activación, parámetro de regularización y tasa de apredizaje.

In [12]:
hyper_param_grid = {'mlp__hidden_layer_sizes': [10, 100,150, 200], 'mlp__activation': ['relu', 'tanh'], 'mlp__alpha': [0.0001, 0.0002, 0.0005, 00.001, 0.005],'mlp__learning_rate_init': [0.001, 0.005, 0.01, 0.05, 0.002]}
my_grid = GridSearchCV(my_pipe, hyper_param_grid, cv = 10, n_jobs=-1)

### Búsqueda de parámetros

Se ajusta el modelo; la búsqueda de los mejores hiperparámetros de manera secuencial tardó 6 minutos con 28 segundos.

In [19]:
%%time
my_grid.fit(x_train, y_train)

CPU times: user 6.03 s, sys: 1.15 s, total: 7.18 s
Wall time: 6min 28s


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('my_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_r...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'mlp__hidden_layer_sizes': [10, 100, 150, 200], 'mlp__learning_rate_init': [0.001, 0.005, 0.01, 0.05, 0.002], 'mlp__alpha': [0.0001, 0.0002, 0.0005, 0.001, 0.005], 'mlp__activation': ['relu', 'tanh']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

Ahora se buscan los hiperparámetros decorando con la función `delayed` para hacer la búsqueda en paralelo. En esta ocasión se ganaron unos segundos en la búsqueda. El tiempo fue de 6 min. 17s.

In [21]:
%%time
delayed(my_grid.fit(x_train, y_train))

CPU times: user 5.89 s, sys: 1.12 s, total: 7.01 s
Wall time: 6min 17s


Delayed('GridSearchCV-6b5ccb63-359b-465a-a8bc-4e9ba7ccad19')

Lamentablemente, el modelo `MLPRegressor` no tiene un análogo en la librería *Dask ML* por lo que la alternativa que utilicé fue correr el modelo de sklearn pero la búsqueda dentro del grid fue utilizando la función `GridSearchCV` proveniente de Dask ML.

In [30]:
from dask_ml.model_selection import GridSearchCV as DaskGridSearch
dk_grid_search = DaskGridSearch(my_pipe, param_grid=hyper_param_grid,cv=10 ,n_jobs=-1)

El tiempo de ejecución fue mayor. Por tanto decidí a ajustar un modelo que se encontrara tanto en sklearn como en DaskML para tener un mejor parámetro de comparación.

In [31]:
%%time
dk_grid_search.fit(x_|train, y_train)

CPU times: user 10min 6s, sys: 20min 23s, total: 30min 29s
Wall time: 8min 31s


GridSearchCV(cache_cv=True, cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('my_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_r...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       iid=True, n_jobs=-1,
       param_grid={'mlp__hidden_layer_sizes': [10, 100, 150, 200], 'mlp__learning_rate_init': [0.001, 0.005, 0.01, 0.05, 0.002], 'mlp__alpha': [0.0001, 0.0002, 0.0005, 0.001, 0.005], 'mlp__activation': ['relu', 'tanh']},
       refit=True, return_train_score='warn', scheduler=None, scoring=None)

## Linear Regression

Al no contar con un modelo `MLPRegressor` en DaskML, decidí por utilizar la función `LinearRegression` que sí está en DASK.

Se ajusta una regresión Ridge y los hiperparámetros a elegir son el nivel `alpha` de regularización y el método de `solver` a utilizar (éste último lo puse como "hiperparámetro" ya que no hay otros de dónde elegir).

In [36]:
from sklearn.linear_model import Ridge
model2 = Ridge(normalize='True')

In [37]:
my_pipe2 = Pipeline([('ridge', model2)])
my_pipe2.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize='True', random_state=None, solver='auto', tol=0.001))])

In [45]:
hyper_param_grid2 = {'ridge__alpha': [0.001, 0.002, 0.0001, 0.0002, 0.0005, 0.01, 0.02], 'ridge__solver': ['svd', 'sag', 'saga']}
my_grid2 = GridSearchCV(my_pipe2, hyper_param_grid2, cv = 10, n_jobs=-1)

Vemos que con `delayed` tardó un poco menos que de manera secuencial.

In [46]:
%%time
my_grid2.fit(x_train, y_train)

CPU times: user 3.33 s, sys: 7.66 s, total: 11 s
Wall time: 3.18 s


GridSearchCV(cache_cv=True, cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize='True', random_state=None, solver='auto', tol=0.001))]),
       iid=True, n_jobs=-1,
       param_grid={'ridge__solver': ['svd', 'sag', 'saga'], 'ridge__alpha': [0.001, 0.002, 0.0001, 0.0002, 0.0005, 0.01, 0.02]},
       refit=True, return_train_score='warn', scheduler=None, scoring=None)

In [47]:
%%time
delayed(my_grid2.fit(x_train, y_train))

CPU times: user 3.33 s, sys: 6.94 s, total: 10.3 s
Wall time: 2.84 s


Delayed('GridSearchCV-22eb6793-a6ce-4305-8b05-896a9bd17fb3')

Definimos el modelo y el `grid` a utilizar para DaskML (regresión lineal).

In [59]:
from dask_ml.linear_model import LinearRegression as LinReg
model2 = LinReg()
my_pipe2 = Pipeline([('my_scaler', StandardScaler()),('ridge', model2)])

In [56]:
my_pipe2.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('my_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('ridge', LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False))])

In [57]:
hyper_param_grid2 = {'ridge__C': [1000, 500, 10000, 5000, 2000, 100, 50], 'ridge__penalty': ['l1', 'l2']}
my_grid2 = GridSearchCV(my_pipe2, hyper_param_grid2, cv = 10, n_jobs=-1)

Tarda menos que de manera secuencial y que con `delayed`. Supongo que esto se debe a que la función está en cierto modo optimizada para realizarse en paralelo. Hubiera sido interesante poder implementar una red neuronal (nativa tanto de sklearn y de DaskMl) con un grid grande de hiperparámetros para observar con mayor claridad el tiempo de ejecución de los modelos.

In [61]:
%%time
my_grid2.fit(x_train, y_train)

CPU times: user 2.73 s, sys: 5.32 s, total: 8.04 s
Wall time: 2.23 s


GridSearchCV(cache_cv=True, cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('my_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('ridge', LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False))]),
       iid=True, n_jobs=-1,
       param_grid={'ridge__penalty': ['l1', 'l2'], 'ridge__C': [1000, 500, 10000, 5000, 2000, 100, 50]},
       refit=True, return_train_score='warn', scheduler=None, scoring=None)